In [1]:
print("Imports")
from keras.models import load_model
from Class_Image import Image
from os.path import abspath
from Class_training import *
from Model_comparisons import *
from os.path import abspath, join
import numpy as np
import requests
import matplotlib.pyplot as plt
#import keras
from tensorflow.keras import Input, Model
import pandas
import Class_Image as Ci
from pymapillary import Mapillary
from PIL import Image
import cv2
import asyncio
%matplotlib inline

import mercantile, mapbox_vector_tile, requests, json
from vt2geojson.tools import vt_bytes_to_geojson
from haversine import haversine
import datetime
import time

Imports


In [9]:
#async 
async def get_image_at_url(path, coords):
    return np.array(Ci.Image(path).preprocess_image(224)).reshape([1,224*224*3]), coords

#async 
def url_at_coords(api, west,north):
    return api.url_at_coords(west, north)

IndentationError: expected an indented block (<ipython-input-9-f5c27e83ddd6>, line 5)

In [3]:

# Convert captured_at timestamp to a date
def msEpochToDate(in_ms_since_epoch):
    s = in_ms_since_epoch / 1000.0
    return(datetime.datetime.fromtimestamp(s).strftime('%Y-%m-%d %H:%M:%S.%f'))

class Mapillary:
    def __init__(self, access_token, search_b_left, search_t_right, zoom = 14, density=200, batch_size=6):
        self.density = density
        self.batch_size = batch_size
        self.search_b_left = search_b_left #Format [W, N]
        self.search_t_right = search_t_right  #Format [W, N]
        self.zoom = zoom
        self.reset()
        
        self.access_token = access_token
        
        # header for call to image endpoint
        self.headers= { "Authorization" : "OAuth {}".format(access_token) }
        
    def reset(self):
        self.tile = mercantile.tile(self.search_b_left[0], self.search_b_left[1], self.zoom) #tile that moves accross map
        
        self.b_left_tile = mercantile.tile(self.search_b_left[0], self.search_b_left[1], self.zoom) #tile at bottom left
        self.t_right_tile = mercantile.tile(self.search_t_right[0], self.search_t_right[1], self.zoom) #tile at top right
    
    #def url_at_coords(self, east, south, west, north, show_progess = False):
    def image_url_generator(self, show_progess = False):

        # define an empty geojson as output
        output= { "type": "FeatureCollection", "features": [] }
        
        # loop through all tiles to get IDs of Mapillary data
        # Input coordinates would be the original point coordinates sent to the class
        
        filter_radius = 200
        
        # get the tiles with x and y coors intersecting bbox at zoom 14 only
        #tiles = list(mercantile.tiles(east, south, west, north, zoom))
        
        v_x = -1 if self.b_left_tile.x > self.t_right_tile.x else 1
        v_y = -1 if self.b_left_tile.y > self.t_right_tile.y else 1
        
        
        for x in range(self.b_left_tile.x, self.t_right_tile.x + v_x, v_x):
            for y in range(self.b_left_tile.y, self.t_right_tile.y + v_y, v_y):
                    tile = mercantile.tile(*mercantile.ul(x, y, self.zoom) + (self.zoom,))
                    #print(mercantile.bounds(x, y, self.zoom))
                   # print(mercantile.ul(tile.x, tile.y, tile.z), tile.x, tile.y)
                   # return
                
                    base = 'https://tiles.mapillary.com/maps/vtp/mly1_public/2'
                    tile_url = f'{base}/{tile.z}/{tile.x}/{tile.y}?access_token={self.access_token}'
                    #print(tile_url)
                    response = requests.get(tile_url)
                    if response.content.startswith(b"<!doctype html>"):
                        print("Mapillary API lockout, trying again in 600 seconds")
                        time.sleep(600)
                        y-= v_y
                        continue
                        
                    data = vt_bytes_to_geojson(response.content, tile.x, tile.y, tile.z)
                    
                    filtered_data = [feature for feature in data['features'] if feature['geometry']['type'] in ['Point']]
                    #TODO sort by date, get summer/winter
                    #summer May 15th - Oct 1st
                    #Winter December - March
                    if(len(filtered_data) == 0):
                        print("empty tile")
                        continue
                    
                    #Get lastest images
                    #filtered_data = sorted(filtered_data, key = lambda k: k['properties']['captured_at'], reverse=True) 
                    filtered_data = filtered_data[0:self.density]
                    
                    fields = ['captured_at','thumb_256_url','geometry']
                    fields = ','.join(fields)
                    
                    for d in filtered_data:
                        
                        image_endpoint_url = f'https://graph.mapillary.com/{d["properties"]["id"]}?fields={fields}&access_token={api.access_token}'
                        image_response = requests.get(image_endpoint_url, headers=self.headers).json()
                        #print(image_response)
                        #print(image_response['thumb_256_url'])
                        if 'thumb_256_url' not in image_response:
                            continue 
                            
                        yield image_response['thumb_256_url'], image_response['geometry']['coordinates']
                    
                    
                    #return
                    
        #print(len(tiles))
        
        
        #return image_dates_urls[0]

In [4]:


df = pandas.DataFrame(columns=[["long", "lat", "rank"]])#.append([["long", "lat", "rank"]])
conv_model = ranking_model(224)
conv_model.load_weights("D:/ranking_model.h5")
model = Model(conv_model.input, conv_model.layers[24].output)
model.training = False
#sample_points = pandas.read_excel("D:/Road_Centrelines_FeatureVert2_TableToExcel.xlsx", sheet_name=None)
#sample_points = sample_points['Road_Centrelines_FeatureVert2_T'][["POINT_X","POINT_Y"]].sample(frac=1) #shuffle


In [5]:

#small testing area
#api = Mapillary('MLY|4459801330748375|f17ddc8a7adbcbff6eea96b1cf4c2aad', [-75.854567689722,45.275097833889], [-75.854558301944,45.300842095])
#Ottawa bounding box
api = Mapillary('MLY|4459801330748375|f17ddc8a7adbcbff6eea96b1cf4c2aad', [-75.9216253, 45.2157973], [-75.4695785, 45.4958371])
#Small Ottawa bounding box
#api = Mapillary('MLY|4459801330748375|f17ddc8a7adbcbff6eea96b1cf4c2aad', [-75.7766456, 45.3224003 ], [-75.6728678, 45.4413471])

df = pandas.DataFrame(columns=[["long", "lat", "rank"]])
save_path = 'D:/coord_rankings_ottawa_2.csv'

gen = api.image_url_generator()

#t = asyncio.create_task(cock())

url_path, coords = next(gen)
img_async = asyncio.create_task(get_image_at_url(url_path, coords))

async def async_gen(gen):
    try:
        return next(gen)
    except StopIteration:
        return False, False

url_async = asyncio.create_task(async_gen(gen))

#for i, (url_path, coords) in enumerate(gen):
i=0
while True:
    i+=1
    
    url_path, coords = await url_async #next(gen)
    #url_path, coords = next(gen)
    if url_path is False: #on generator end
        break
        
    url_async = asyncio.create_task(async_gen(gen))

    
    img, coords_ = await img_async#get_image_at_url(url_path, coords)
    img_async = asyncio.create_task(get_image_at_url(url_path, coords))
    #img, coords_ = await get_image_at_url(url_path, coords)

    df.loc[i+1] = [coords_[0], coords_[1], np.array(model([img, img]))[0][0]] 
    
    if i % 500 == 0:
        df.to_csv(save_path, index=False)
    print(f"\r Processing image {i} | {coords_[0]} {coords_[1]}", end="")
    
df.to_csv(save_path, index=False)


 Processing image 173 | -75.924249603904 45.224066726779

CancelledError: 

In [7]:
df.loc[1:4] = [[3,3,3],[3,3,3],[3,3,3]]

In [8]:
df

,long,lat,rank
2,3.000000,3.000000,3.000000
3,3.000000,3.000000,3.000000
4,3.000000,3.000000,3.000000
5,-75.927012,45.222391,0.136088
6,-75.925903,45.223053,-1.156369
...,...,...,...
170,-75.932003,45.219345,-1.186592
171,-75.920602,45.226395,-1.017232
172,-75.926018,45.223102,0.060746
173,-75.925204,45.223592,-0.300034
